Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain, JLD
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df);
first(df2, 5)

,height,weight,age,male
,Float64⍰,Float64⍰,Float64⍰,Int64⍰
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0


Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height],
  "weight" => df2[:weight]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME)
# Describe the draws
describe(chn)

=====> /Users/rob/.julia/dev/StatisticalRethinking/scripts/04


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan will be updated.



--- Translating Stan model to C++ code ---
bin/stanc  --o=/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.hpp /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan
Model name=weights_model
Input file=/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan
Output file=/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.hpp
clang++ -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare      -O3 -I src -I stan/src -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.3 -I stan/lib/stan_math/lib/boost_1.66.0 -I stan/lib/stan_math/lib/sundials_3.1.0/include    -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION     -c -MT /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/t

Save the chains in a JLD file

In [7]:
serialize("m4.3s.jls", chn)

chn2 = deserialize("m4.3s.jls")

describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 113.84444900 1.951895071 0.0308621709 0.0525348356 1000
 beta   0.90576583 0.042887873 0.0006781168 0.0011577027 1000
sigma   5.10659623 0.189462878 0.0029956711 0.0039908975 1000

Quantiles:
          2.5%        25.0%       50.0%        75.0%       97.5%   
alpha 110.0999750 112.53300000 113.8170000 115.15250000 117.8300250
 beta   0.8190900   0.87663875   0.9071095   0.93477875   0.9870114
sigma   4.7557388   4.97052250   5.1004950   5.23454000   5.4873388



Should be identical to earlier result

In [8]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 113.84444900 1.951895071 0.0308621709 0.0525348356 1000
 beta   0.90576583 0.042887873 0.0006781168 0.0011577027 1000
sigma   5.10659623 0.189462878 0.0029956711 0.0039908975 1000

Quantiles:
          2.5%        25.0%       50.0%        75.0%       97.5%   
alpha 110.0999750 112.53300000 113.8170000 115.15250000 117.8300250
 beta   0.8190900   0.87663875   0.9071095   0.93477875   0.9870114
sigma   4.7557388   4.97052250   5.1004950   5.23454000   5.4873388



End of `m4.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*